In [1]:
from PIL import Image
import sys, os
import numpy as np
import imageio
from scipy import ndimage


In [13]:
img_list = []
root = "frames"
root_seq = os.path.join(root, "sequence")
states = ["slash", "spellcard", "walk"]
sides = ["front", "left", "right"]

if not os.path.exists(root_seq):
    os.makedirs(root_seq)

for n in range(6):
    for state in states: 
        for side in sides:
            sequence = Image.new("RGBA", (64*8, 64))
            for i in range(8):
                img_file = f"{side}_{n:04d}_{i}.png"
                img_path = os.path.join(root, state, img_file);
                frame = Image.open(img_path);
                sequence.paste(frame, (i * 64, 0))
            seq_path = os.path.join(root_seq, f"{n:04d}_{state}_{side}.png");
            sequence.save(seq_path)
            

saving as frames/sequence/0000_slash_front.png
saving as frames/sequence/0000_slash_left.png
saving as frames/sequence/0000_slash_right.png
saving as frames/sequence/0000_spellcard_front.png
saving as frames/sequence/0000_spellcard_left.png
saving as frames/sequence/0000_spellcard_right.png
saving as frames/sequence/0000_walk_front.png
saving as frames/sequence/0000_walk_left.png
saving as frames/sequence/0000_walk_right.png
saving as frames/sequence/0001_slash_front.png
saving as frames/sequence/0001_slash_left.png
saving as frames/sequence/0001_slash_right.png
saving as frames/sequence/0001_spellcard_front.png
saving as frames/sequence/0001_spellcard_left.png
saving as frames/sequence/0001_spellcard_right.png
saving as frames/sequence/0001_walk_front.png
saving as frames/sequence/0001_walk_left.png
saving as frames/sequence/0001_walk_right.png
saving as frames/sequence/0002_slash_front.png
saving as frames/sequence/0002_slash_left.png
saving as frames/sequence/0002_slash_right.png
sa